### Import libraries

In [26]:
import pandas as pd
import plotly.express as px

### Read Data

In [110]:
itinerarios_bases = pd.read_csv('../Data/Itinerarios/itinerarios_bases.csv',index_col=0, dtype={'zip_code': 'str'})
itinerarios_bases.dtypes

idplug_base                   int64
user_type                     int64
idunplug_base                 int64
travel_time                   int64
idunplug_station              int64
ageRange                      int64
idplug_station                int64
unplug_hourTime              object
zip_code                     object
return_date                  object
status                       object
Origen_destino               object
Número_Salida                 int64
Distrito_Salida              object
Barrio_Salida                object
Número de Plazas_Salida     float64
Latitud_Salida              float64
Longitud_Salida             float64
Número_Llegada                int64
Distrito_Llegada             object
Barrio_Llegada               object
Número de Plazas_Llegada    float64
Latitud_Llegada             float64
Longitud_Llegada            float64
dtype: object

### Reshape data

In [112]:
#df_combinations = itinerarios_bases.groupby(["Origen_destino","idunplug_station", "Distrito_Llegada", "idplug_station"])\
#    .size().reset_index().rename(columns={0:"count"})
#df_combinations.head(10)

,Origen_destino,idunplug_station,Distrito_Llegada,idplug_station,count
0,1-1,1,01 CENTRO,1,270
1,1-10,1,01 CENTRO,10,145
2,1-100,1,04 SALAMANCA,100,9
3,1-101,1,04 SALAMANCA,101,28
4,1-102,1,04 SALAMANCA,102,42
5,1-103,1,04 SALAMANCA,103,76
6,1-104,1,04 SALAMANCA,104,29
7,1-105,1,04 SALAMANCA,105,30
8,1-106,1,04 SALAMANCA,106,55
9,1-107,1,04 SALAMANCA,107,35


Filter the top N stations with more unplug registrations.

In [124]:
N = int(input("¿Cuántas estaciones de origen quieres mostrar?: "))
print(N)

¿Cuántas estaciones de origen quieres mostrar?: 5
5


In [125]:
estaciones_mas_concurridas = itinerarios_bases.groupby(["idunplug_station"])\
    .size().reset_index().rename(columns={0:"count"}).sort_values(by=['count'], ascending = False).head(N).reset_index(drop=True)
estaciones_mas_concurridas1 = itinerarios_bases[itinerarios_bases["idunplug_station"].isin(estaciones_mas_concurridas["idunplug_station"])]

df_combinations = estaciones_mas_concurridas1.groupby(["idunplug_station", "Distrito_Llegada", "idplug_station"])\
    .size().reset_index().rename(columns={0:"count"}).sort_values(by=['count'], ascending = False)
df_combinations.head(10)

,idunplug_station,Distrito_Llegada,idplug_station,count
909,175,02 ARGANZUELA,175,1010
34,43,01 CENTRO,43,893
255,57,01 CENTRO,57,887
855,175,01 CENTRO,27,851
512,83,03 RETIRO,83,782
870,175,01 CENTRO,43,754
238,57,01 CENTRO,38,656
73,43,02 ARGANZUELA,175,631
1016,175,07 CHAMBERÍ,135,616
689,163,02 ARGANZUELA,163,613


In [126]:
M = int(input("¿Cuántas estaciones de salida quieres mostrar por estación de origen?: "))
print(M)

¿Cuántas estaciones de salida quieres mostrar por estación de origen?: 10
10


In [127]:
topCombinaciones = pd.DataFrame()
for i in df_combinations["idunplug_station"].unique():
    estacion_origen = df_combinations[df_combinations["idunplug_station"] == i]
    topCombinacionesi = estacion_origen.sort_values(by=['count'], ascending = False).head(M)
    topCombinaciones = topCombinaciones.append(topCombinacionesi)
topCombinaciones.reset_index(drop=True)
topCombinaciones.head()

<ipython-input-127-73eef84b1126>:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-127-73eef84b1126>:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-127-73eef84b1126>:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-127-73eef84b1126>:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-127-73eef84b1126>:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,idunplug_station,Distrito_Llegada,idplug_station,count
909,175,02 ARGANZUELA,175,1010
855,175,01 CENTRO,27,851
870,175,01 CENTRO,43,754
1016,175,07 CHAMBERÍ,135,616
1014,175,07 CHAMBERÍ,132,596


In [130]:
fig = px.sunburst(
    topCombinaciones,
    path = ["idunplug_station", "Distrito_Llegada", "idplug_station"],
    title = "Distribución rutas más concurridas", 
    color = "idunplug_station",
    height=800,
    values='count',
    color_continuous_scale='RdBu',
)
fig.show()